# import modules

In [96]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
# from sklearn.pipeline import Pipeline
import datetime
import time

# Data set 불러오기

In [97]:
# 여러분의 폴더가 저장되어있는 위치 경로를 써 주세요
# 그리고 데이터파일(주최측에서 제공한) 의 이름을 Train set 으로 변셩해 주세요
# 한인욱 directory = 'C:/Users/goran/Desktop/Git_Hub/Project/20S_Bigcon'
# 김정현 directory = 'C:/Users/Coby/Desktop/20S_Bigcon/20S_Bigcon'
# 신예진 directory = 'C:/Users/Rstudio/Desktop/Bigcon2020/20S_Bigcon'
# 이규민 directory = 'C:/Statistics/20S_Bigcon'
# 최정욱 directory = 'C:/Users/lg/Desktop/Bigcon2020'
directory = 'C:/Statistics/20S_Bigcon'

In [98]:
# 그러면 이렇게 그 폴더 안의 Train set.xlsx 을 잘 읽어옵니다! 
X_df=pd.read_excel(directory+'/Train set.xlsx',header = 1) 

In [99]:
# 데이터 reset 하고싶으면 위의 셀 말고 이거 돌려가면서 reset 하세요! 그게 제일 빨라요
X = X_df.copy()

In [100]:
X.head() 

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0


# 결측치 처리 및 변수늘리기

In [101]:
X.isnull().sum()

방송일시         0
노출(분)    16784
마더코드         0
상품코드         0
상품명          0
상품군          0
판매단가         0
취급액        937
dtype: int64

## 노출(분) 결측치 처리

노출분은 같은 시간에 같이 방송되어서 NaN 으로 뜨는것이다. <br>
아마 같은 시간에 방송되어진 값을 평균해서 채우면 될듯?

In [102]:
X.loc[:,'평균방송분'] = X.loc[:,'노출(분)']

In [103]:
# 0 노출시간이 0 인 데이터가 없다.
# 즉 Na 처리할떄 0으로 대채한 후에 평균떄려도 아무 상관 없다는 이야기!
(X.loc[:,'노출(분)']==0).sum()

0

In [104]:
# 노출(분) 을 제일 먼저 시작한 시간의 값으로 만드는 FUNCTION 정의
# 아직 맞는지는 잘 몰라서 안맞는 경우 수정해주세요.
def NA_minute(df=X):
    # 곁측치를 0으로 대체
    df['노출(분)'] = df['노출(분)'].fillna(0)
    
    count_0 = [] 
    count_not0 = [] # 0이 아닌 수의 index
    for idx in df.index :
        if df.loc[idx,'노출(분)'] == 0 :
            count_0.append(idx)
        else :
            count_not0.append(idx)
    count_not0.append(len(df)) # 맨 마지막 열에 0 인 경우를 대비하여

    for idx,x in enumerate(count_not0):
        if idx != len(count_not0)-1: 
            if (count_not0[idx+1] - count_not0[idx] - 1) != 0:
                start = count_not0[idx] 
                end = count_not0[idx+1]
                num = end - start 
                df.loc[start:(end-1),'노출(분)'] = df.loc[start,'노출(분)'] # / num 을 붙여주면 평균으로 만들게 됩니다.
    return df

In [105]:
X = NA_minute(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분
0,2019-01-01 06:00:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,20.000000
1,2019-01-01 06:00:00,20.000000,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,NaN
2,2019-01-01 06:20:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,20.000000
3,2019-01-01 06:20:00,20.000000,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,NaN
4,2019-01-01 06:40:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,20.000000
...,...,...,...,...,...,...,...,...,...
38304,2020-01-01 00:20:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000
38305,2020-01-01 00:40:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000
38306,2020-01-01 01:00:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000
38307,2020-01-01 01:20:00,20.000000,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,NaN,20.000000


## 얼마나 같은 방송일시에 겹쳐있는지 변수늘리기

In [106]:
X.loc[:,'겹치는수'] = 1

In [107]:
for idx in range(1,len(X))  :
    if ~(X.loc[idx-1,'마더코드'] == X.loc[idx,'마더코드']): # 앞과 마더코드가 다를때!
        continue # 아무 관련 없으므로 다음 for 문으로 넘어가도록 합시다
    elif (X.loc[idx-1,'마더코드'] == X.loc[idx,'마더코드']) : # 앞 구간과 같아버리면 같은게 지속되고있는거겟죠!
        if X.loc[idx-1,'방송일시'] == X.loc[idx,'방송일시']: # 만약 전의 방송일시와 지금이 일치하면 그냥 전 index 의 겹치는 수를 씁니다. 
            X.loc[idx,'겹치는수'] = X.loc[idx-1,'겹치는수']
        elif ~(X.loc[idx-1,'방송일시'] == X.loc[idx,'방송일시']): # 만약 전의 방송일시와 지금이 다르다면 , 마더코드는 같은데 방송일시가 다르단거겟죠? index 를 증가시킵니다.
            X.loc[idx,'겹치는수'] = X.loc[idx-1,'겹치는수']+1

In [108]:
X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수
0,2019-01-01 06:00:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,20.000000,1
1,2019-01-01 06:00:00,20.000000,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,NaN,1
2,2019-01-01 06:20:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,20.000000,2
3,2019-01-01 06:20:00,20.000000,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,NaN,2
4,2019-01-01 06:40:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,20.000000,3
...,...,...,...,...,...,...,...,...,...,...
38304,2020-01-01 00:20:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000,1
38305,2020-01-01 00:40:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000,2
38306,2020-01-01 01:00:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000,3
38307,2020-01-01 01:20:00,20.000000,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,NaN,20.000000,1


## 평균 방송분 변수늘리기

In [109]:
def MEAN_minute(df=X):
    # 곁측치를 0으로 대체
    df['평균방송분'] = df['평균방송분'].fillna(0)
    
    count_0 = [] 
    count_not0 = [] # 0이 아닌 수의 index
    for idx in df.index :
        if df.loc[idx,'평균방송분'] == 0 :
            count_0.append(idx)
        else :
            count_not0.append(idx)
    count_not0.append(len(df)) # 맨 마지막 열에 0 인 경우를 대비하여

    for idx,x in enumerate(count_not0):
        if idx != len(count_not0)-1: 
            if (count_not0[idx+1] - count_not0[idx] - 1) != 0:
                start = count_not0[idx] 
                end = count_not0[idx+1]
                num = end - start 
                df.loc[start:(end-1),'평균방송분'] = df.loc[start,'평균방송분'] / num
    return df

In [110]:
X = MEAN_minute(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수
0,2019-01-01 06:00:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.000000,1
1,2019-01-01 06:00:00,20.000000,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.000000,1
2,2019-01-01 06:20:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.000000,2
3,2019-01-01 06:20:00,20.000000,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.000000,2
4,2019-01-01 06:40:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.000000,3
...,...,...,...,...,...,...,...,...,...,...
38304,2020-01-01 00:20:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000,1
38305,2020-01-01 00:40:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000,2
38306,2020-01-01 01:00:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000,3
38307,2020-01-01 01:20:00,20.000000,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,NaN,20.000000,1


## 시간 데이터 늘리기

In [111]:
# 시각은 우선 시,분을 합쳐서 하나의 값으로 만들었습니다.
def time_feature(df = X):
    df['방송일시']=df['방송일시'].astype('str')
    df['날짜']=df['방송일시'].str.split(' ').str[0]
    df['시간']=df['방송일시'].str.split(' ').str[1]
    df['년']=df['날짜'].str.split('-').str[0]
    df['월']=df['날짜'].str.split('-').str[1]
    df['일']=df['날짜'].str.split('-').str[2]
    df['시']=df['시간'].str.split(':').str[0]
    df['분']=df['시간'].str.split(':').str[1]
    df['초']=df['시간'].str.split(':').str[2]
    df['시각'] = df['시'].astype('float32') + df['분'].astype('float32')*(1/60)
    df['시분'] = df['시'] + ':' + df['분']
    df.drop(['시','분','초','시간'],axis=1,inplace = True)
    return(df)

In [112]:
X = time_feature(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,날짜,년,월,일,시각,시분
0,2019-01-01 06:00:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.000000,1,2019-01-01,2019,01,01,6.000000,06:00
1,2019-01-01 06:00:00,20.000000,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.000000,1,2019-01-01,2019,01,01,6.000000,06:00
2,2019-01-01 06:20:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.000000,2,2019-01-01,2019,01,01,6.333333,06:20
3,2019-01-01 06:20:00,20.000000,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.000000,2,2019-01-01,2019,01,01,6.333333,06:20
4,2019-01-01 06:40:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.000000,3,2019-01-01,2019,01,01,6.666667,06:40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38304,2020-01-01 00:20:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000,1,2020-01-01,2020,01,01,0.333333,00:20
38305,2020-01-01 00:40:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000,2,2020-01-01,2020,01,01,0.666667,00:40
38306,2020-01-01 01:00:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000,3,2020-01-01,2020,01,01,1.000000,01:00
38307,2020-01-01 01:20:00,20.000000,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,NaN,20.000000,1,2020-01-01,2020,01,01,1.333333,01:20


## 주말 시청률 평균 생성

주파수를 사용하지 않는 케이블, 위성방송, IPTV의 등장에 따라 셋톱박스에서 조사가 가정해져서 시청률이 좀 더 정확해졌으며, 시청률 조사지역은 점차 확대되어 수도권, 5대 광역시, 중소도시등이 포함되어 2019년에는 모든 케이블 방송권역에서 조사가 이루어지고 있다. 2020년 현재 TNMS는 전국(제주도 및 읍·면 지역 포함) 4,000여 가구에 거주하는 약 9천명을 셋탑박스를 조사(TMNS 홈페이지 기준), 닐슨 코리아에서는 피플미터기로 4,220가구(닐슨 컴퍼니 코리아 홈페이지 기준)를 조사한다.

+) http://www.mediaus.co.kr/news/articleView.html?idxno=22809

In [113]:
# 앞의 과정처럼 그 폴더 안의 Train set.xlsx 을 읽습니다. 
rate_df=pd.read_excel(directory+'/Train_rating.xlsx',header = 1) 

In [114]:
# 데이터 reset 하고싶으면 위의 셀 말고 이거 돌려가면서 reset 합시다.(앞과 같은 과정입니다)
rate = rate_df.copy()

In [115]:
time_list= ['0101','0104','0105','0106','0111','0112','0113','0118','0119','0120','0125','0126','0127',
                  '0201','0202','0203','0204','0205','0206','0208','0209','0210','0215','0216','0217','0222','0223','0224',
                  '0301','0302','0303','0308','0309','0310','0315','0316','0317','0322','0323','0324','0329','0330','0331',
                  '0405','0406','0407','0412','0413','0414','0419','0420','0421','0426','0427','0428'
                  ,'0503','0504','0505','0506','0510','0511','0512','0517','0518','0519','0524','0525','0526','0531',
            '0601','0602','0606','0607','0608','0609','0614','0615','0616','0621','0622','0623','0628','0629','0630',
            '0704','0705','0706','0707','0712','0713','0714','0719','0720','0721','0726','0727','0728',
            '0802','0803','0804','0809','0810','0811','0815','0816','0817','0818','0823','0824','0825',
            '0901','0906','0907','0908','0912','0913','0914','0915','0920','0921','0922','0927','0928','0929'
 ,'1003','1004','1005','1006','1009','1011','1012','1013','1018','1019','1020','1025','1026','1027',
                '1101','1102','1103','1108','1109','1110','1115','1116','1117','1122','1123','1124','1129','1130',
                  '1201','1206','1207','1208','1213','1214','1215','1220','1221','1222','1225','1227','1228','1229']

In [116]:
rate_T = rate.T  # TRANSPOSE 를 해야 알맞게 변환이 됩니다.
rate_T.reset_index(inplace=True)
rate_T.columns = rate_T.iloc[0,:]
rate_T.drop('월화수목금토일02:00-01:59',axis=1,inplace=True)
rate_T.drop(index=366,inplace=True)
rate_T.drop(index=0,inplace=True)
rate_T.loc[:,'시간대'] = rate_T.loc[:,'시간대'].astype(str)
rate_T['월']=rate_T['시간대'].str.split('-').str[1]
rate_T['일']=rate_T['시간대'].str.split('-').str[2]
rate_T['월일'] = rate_T['월'].astype(str) + rate_T['일'].astype(str)
rate_T.reset_index(drop=True,inplace = True)

In [117]:
for i in range(len(rate_T)):
    if rate_T.loc[i,'월일'] not in time_list: # 주말이 아니라면, drop 시킬거야! 
        rate_T.drop(index = i , inplace = True) # 그러면 주말만 남을겁니다.

In [118]:
rate_T

,시간대,02:00,02:01,02:02,02:03,02:04,02:05,02:06,02:07,02:08,...,01:53,01:54,01:55,01:56,01:57,01:58,01:59,월,일,월일
0,2019-01-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,01,01,0101
3,2019-01-04,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,01,04,0104
4,2019-01-05,0,0,0,0,0,0,0,0,0,...,0,0,0,0.027,0,0,0,01,05,0105
5,2019-01-06,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,01,06,0106
10,2019-01-11,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.009,0.009,0.014,01,11,0111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,2019-12-22,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,12,22,1222
358,2019-12-25,0,0,0,0.017,0.017,0.017,0.017,0.017,0.017,...,0,0,0,0,0,0,0,12,25,1225
360,2019-12-27,0,0,0,0,0,0,0,0,0,...,0.015,0.015,0.015,0.015,0.015,0.015,0.015,12,27,1227
361,2019-12-28,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,12,28,1228


In [119]:
rate_T.drop(columns=['월','일','월일'],inplace= True)

rate_T.drop(columns = ['시간대'],inplace= True)

rate_T=rate_T.astype(float)

weekend=pd.DataFrame(np.mean(rate_T,axis=0),columns=['시청률 평균'])

weekend.index.name = None

In [120]:
weekend

,시청률 평균
02:00,0.001843
02:01,0.001928
02:02,0.002392
02:03,0.002518
02:04,0.002590
...,...
01:55,0.003693
01:56,0.003554
01:57,0.003627
01:58,0.003410


In [121]:
weekend=pd.DataFrame(np.mean(rate_T,axis=0),columns=['시청률 평균'])

weekend.index.name = '시간'

weekend=weekend.reset_index()

weekend=pd.concat([weekend.loc[240:],weekend.loc[:239]],axis=0)

weekend=weekend.reset_index(drop=True)

In [122]:
weekend

,시간,시청률 평균
0,06:00,0.002175
1,06:01,0.002434
2,06:02,0.002747
3,06:03,0.003976
4,06:04,0.003795
...,...,...
1435,05:55,0.002018
1436,05:56,0.002819
1437,05:57,0.002608
1438,05:58,0.002753


In [123]:
weekend.loc[:,'시간'] = weekend.loc[:,'시간'].astype(str)
weekend['시각']=weekend['시간'].str.split(':').str[0].astype(str) + weekend['시간'].str.split(':').str[1].astype(str)

weekend['시각']=weekend['시각'].astype(int)

for i in range(len(weekend)):
    if weekend.loc[i,'시각'] < 600:
         weekend.loc[i,'시각']  += 2400

In [124]:
weekend

,시간,시청률 평균,시각
0,06:00,0.002175,600
1,06:01,0.002434,601
2,06:02,0.002747,602
3,06:03,0.003976,603
4,06:04,0.003795,604
...,...,...,...
1435,05:55,0.002018,2955
1436,05:56,0.002819,2956
1437,05:57,0.002608,2957
1438,05:58,0.002753,2958


In [125]:
weekend.index = weekend['시각']

In [126]:
weekend

,시간,시청률 평균,시각
시각,,,
600,06:00,0.002175,600
601,06:01,0.002434,601
602,06:02,0.002747,602
603,06:03,0.003976,603
604,06:04,0.003795,604
...,...,...,...
2955,05:55,0.002018,2955
2956,05:56,0.002819,2956
2957,05:57,0.002608,2957


## 평일시청률 평균 생성

주파수를 사용하지 않는 케이블, 위성방송, IPTV의 등장에 따라 셋톱박스에서 조사가 가정해져서 시청률이 좀 더 정확해졌으며, 시청률 조사지역은 점차 확대되어 수도권, 5대 광역시, 중소도시등이 포함되어 2019년에는 모든 케이블 방송권역에서 조사가 이루어지고 있다. 2020년 현재 TNMS는 전국(제주도 및 읍·면 지역 포함) 4,000여 가구에 거주하는 약 9천명을 셋탑박스를 조사(TMNS 홈페이지 기준), 닐슨 코리아에서는 피플미터기로 4,220가구(닐슨 컴퍼니 코리아 홈페이지 기준)를 조사한다.

+) http://www.mediaus.co.kr/news/articleView.html?idxno=22809

In [127]:
# 앞의 과정처럼 그 폴더 안의 Train set.xlsx 을 읽습니다. 
rate_df=pd.read_excel(directory+'/Train_rating.xlsx',header = 1) 

In [128]:
# 데이터 reset 하고싶으면 위의 셀 말고 이거 돌려가면서 reset 합시다.(앞과 같은 과정입니다)
rate = rate_df.copy()

In [129]:
time_list= ['0101','0104','0105','0106','0111','0112','0113','0118','0119','0120','0125','0126','0127',
                  '0201','0202','0203','0204','0205','0206','0208','0209','0210','0215','0216','0217','0222','0223','0224',
                  '0301','0302','0303','0308','0309','0310','0315','0316','0317','0322','0323','0324','0329','0330','0331',
                  '0405','0406','0407','0412','0413','0414','0419','0420','0421','0426','0427','0428'
                  ,'0503','0504','0505','0506','0510','0511','0512','0517','0518','0519','0524','0525','0526','0531',
            '0601','0602','0606','0607','0608','0609','0614','0615','0616','0621','0622','0623','0628','0629','0630',
            '0704','0705','0706','0707','0712','0713','0714','0719','0720','0721','0726','0727','0728',
            '0802','0803','0804','0809','0810','0811','0815','0816','0817','0818','0823','0824','0825',
            '0901','0906','0907','0908','0912','0913','0914','0915','0920','0921','0922','0927','0928','0929'
 ,'1003','1004','1005','1006','1009','1011','1012','1013','1018','1019','1020','1025','1026','1027',
                '1101','1102','1103','1108','1109','1110','1115','1116','1117','1122','1123','1124','1129','1130',
                  '1201','1206','1207','1208','1213','1214','1215','1220','1221','1222','1225','1227','1228','1229']

In [130]:
rate_T = rate.T  # TRANSPOSE 를 해야 알맞게 변환이 됩니다.

rate_T.reset_index(inplace=True)

rate_T.columns = rate_T.iloc[0,:]

rate_T.drop('월화수목금토일02:00-01:59',axis=1,inplace=True)

rate_T.drop(index=366,inplace=True)

rate_T.drop(index=0,inplace=True)

rate_T.loc[:,'시간대'] = rate_T.loc[:,'시간대'].astype(str)

rate_T['월']=rate_T['시간대'].str.split('-').str[1]
rate_T['일']=rate_T['시간대'].str.split('-').str[2]

rate_T['월일'] = rate_T['월'].astype(str) + rate_T['일'].astype(str)

rate_T.reset_index(drop=True,inplace = True)

In [131]:
for i in range(len(rate_T)):
    if rate_T.loc[i,'월일'] in time_list: # 주말이 아니라면, drop 시킬거야! 
        rate_T.drop(index = i , inplace = True) # 그러면 주말만 남을겁니다.

rate_T.drop(columns=['월','일','월일'],inplace= True)

rate_T.drop(columns = ['시간대'],inplace= True)

rate_T=rate_T.astype(float)

weekday=pd.DataFrame(np.mean(rate_T,axis=0),columns=['시청률 평균'])

weekday.index.name = '시간'

weekday=weekday.reset_index()

weekday=pd.concat([weekday.loc[240:],weekday.loc[:239]],axis=0)

weekday

,시간,시청률 평균
240,06:00,0.001407
241,06:01,0.001648
242,06:02,0.001492
243,06:03,0.001658
244,06:04,0.001849
...,...,...
235,05:55,0.001136
236,05:56,0.001231
237,05:57,0.000950
238,05:58,0.001286


In [132]:
weekday=weekday.reset_index(drop=True)

In [133]:
weekday

,시간,시청률 평균
0,06:00,0.001407
1,06:01,0.001648
2,06:02,0.001492
3,06:03,0.001658
4,06:04,0.001849
...,...,...
1435,05:55,0.001136
1436,05:56,0.001231
1437,05:57,0.000950
1438,05:58,0.001286


In [134]:
weekday.loc[:,'시간'] = weekday.loc[:,'시간'].astype(str)
weekday['시각']=weekday['시간'].str.split(':').str[0].astype(str) + weekday['시간'].str.split(':').str[1].astype(str)

weekday['시각']=weekday['시각'].astype(int)

for i in range(len(weekday)):
    if weekday.loc[i,'시각'] < 600:
         weekday.loc[i,'시각']  += 2400

weekday

,시간,시청률 평균,시각
0,06:00,0.001407,600
1,06:01,0.001648,601
2,06:02,0.001492,602
3,06:03,0.001658,603
4,06:04,0.001849,604
...,...,...,...
1435,05:55,0.001136,2955
1436,05:56,0.001231,2956
1437,05:57,0.000950,2957
1438,05:58,0.001286,2958


In [135]:
weekday.index = weekday['시각']

weekday

,시간,시청률 평균,시각
시각,,,
600,06:00,0.001407,600
601,06:01,0.001648,601
602,06:02,0.001492,602
603,06:03,0.001658,603
604,06:04,0.001849,604
...,...,...,...
2955,05:55,0.001136,2955
2956,05:56,0.001231,2956
2957,05:57,0.000950,2957


## 시청률 적용

- 금토일,공휴일 과  
- 월화수목 의 시청률 평균을 다르게 계산하였습니다.

In [136]:
X['월일'] = X['월'].astype(str) + X['일'].astype(str)

In [137]:
X.loc[:,'시분'] = X.loc[:,'시분'].astype(str)
X['시간열']=X['시분'].str.split(':').str[0].astype(str) + X['시분'].str.split(':').str[1].astype(str)
X['시간열']=X['시간열'].astype(int)

In [138]:
for i in range(len(X)):
    if X.loc[i,'시간열'] < 600:
         X.loc[i,'시간열']  += 2400

In [139]:
X['정수노출(분)']=X['노출(분)'].round(0).astype(int)

In [140]:
X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,날짜,년,월,일,시각,시분,월일,시간열,정수노출(분)
0,2019-01-01 06:00:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.000000,1,2019-01-01,2019,01,01,6.000000,06:00,0101,600,20
1,2019-01-01 06:00:00,20.000000,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.000000,1,2019-01-01,2019,01,01,6.000000,06:00,0101,600,20
2,2019-01-01 06:20:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.000000,2,2019-01-01,2019,01,01,6.333333,06:20,0101,620,20
3,2019-01-01 06:20:00,20.000000,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.000000,2,2019-01-01,2019,01,01,6.333333,06:20,0101,620,20
4,2019-01-01 06:40:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.000000,3,2019-01-01,2019,01,01,6.666667,06:40,0101,640,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38304,2020-01-01 00:20:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000,1,2020-01-01,2020,01,01,0.333333,00:20,0101,2420,20
38305,2020-01-01 00:40:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000,2,2020-01-01,2020,01,01,0.666667,00:40,0101,2440,20
38306,2020-01-01 01:00:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000,3,2020-01-01,2020,01,01,1.000000,01:00,0101,2500,20
38307,2020-01-01 01:20:00,20.000000,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,NaN,20.000000,1,2020-01-01,2020,01,01,1.333333,01:20,0101,2520,20


In [141]:
X['시청률평균'] = 0

In [142]:
for idx in range(len(X)-1):
    if X.loc[idx,'월일'] in time_list:
        if X.loc[idx,'시간열'] == X.loc[idx+1,'시간열']: # 앗 이경우에는 뒤의값과 같네요. 지속입니다
            X.loc[idx,'시청률평균'] = 1
        elif X.loc[idx,'시간열'] > X.loc[idx+1,'시간열'] : # 앗 이경우에는 전값이 더 크네요? 그 경우에는 날짜가 바뀐것입니다.
             X.loc[idx,'시청률평균'] = np.mean(weekend.loc[X.loc[idx,'시간열']:(X.loc[idx,'시간열']+X.loc[idx,'정수노출(분)']),'시청률 평균'])
        # 이 경우에는 얼마만큼 노출되었는지를 고려해서 그만큼을 MEAN 을 시켜줍니다. 
        elif X.loc[idx,'시간열'] < X.loc[idx+1,'시간열'] : # 뒤의 값이 더 크네요? 그 경우에는 상품이 바뀐것이죠.
             X.loc[idx,'시청률평균'] = np.mean(weekend.loc[X.loc[idx,'시간열']:(X.loc[idx,'시간열']+X.loc[idx,'정수노출(분)']),'시청률 평균'])
    else:
        if X.loc[idx,'시간열'] == X.loc[idx+1,'시간열']: # 앗 이경우에는 뒤의값과 같네요. 지속입니다
            X.loc[idx,'시청률평균'] = 1
        elif X.loc[idx,'시간열'] > X.loc[idx+1,'시간열'] : # 앗 이경우에는 전값이 더 크네요? 그 경우에는 날짜가 바뀐것입니다.
             X.loc[idx,'시청률평균'] = np.mean(weekday.loc[X.loc[idx,'시간열']:(X.loc[idx,'시간열']+X.loc[idx,'정수노출(분)']),'시청률 평균'])
        # 이 경우에는 얼마만큼 노출되었는지를 고려해서 그만큼을 MEAN 을 시켜줍니다. 
        elif X.loc[idx,'시간열'] < X.loc[idx+1,'시간열'] : # 뒤의 값이 더 크네요? 그 경우에는 상품이 바뀐것이죠.
             X.loc[idx,'시청률평균'] = np.mean(weekday.loc[X.loc[idx,'시간열']:(X.loc[idx,'시간열']+X.loc[idx,'정수노출(분)']),'시청률 평균'])

In [143]:
# 마지막값은 그냥 무조건 값이 있어야겟죠?
idx = len(X)-1
if X.loc[idx,'월일'] in time_list:
    X.loc[idx,'시청률평균'] = np.mean(weekend.loc[X.loc[idx,'시간열']:(X.loc[idx,'시간열']+X.loc[idx,'정수노출(분)']),'시청률 평균'])
else:
    X.loc[idx,'시청률평균'] = np.mean(weekday.loc[X.loc[idx,'시간열']:(X.loc[idx,'시간열']+X.loc[idx,'정수노출(분)']),'시청률 평균'])

In [144]:
for idx in range(len(X)-1,-1,-1): # index 를 reverse 로 훑어들어오는것입니다. 
    if X.loc[idx,'시청률평균'] == 1: # 1이란것은 뒤의것과 겹친다는 이야기입니다.
        X.loc[idx,'시청률평균'] = X.loc[idx+1,'시청률평균'] 

In [145]:
X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,날짜,년,월,일,시각,시분,월일,시간열,정수노출(분),시청률평균
0,2019-01-01 06:00:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.000000,1,2019-01-01,2019,01,01,6.000000,06:00,0101,600,20,0.002632
1,2019-01-01 06:00:00,20.000000,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.000000,1,2019-01-01,2019,01,01,6.000000,06:00,0101,600,20,0.002632
2,2019-01-01 06:20:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.000000,2,2019-01-01,2019,01,01,6.333333,06:20,0101,620,20,0.002434
3,2019-01-01 06:20:00,20.000000,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.000000,2,2019-01-01,2019,01,01,6.333333,06:20,0101,620,20,0.002434
4,2019-01-01 06:40:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.000000,3,2019-01-01,2019,01,01,6.666667,06:40,0101,640,20,0.003162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38304,2020-01-01 00:20:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000,1,2020-01-01,2020,01,01,0.333333,00:20,0101,2420,20,0.004859
38305,2020-01-01 00:40:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000,2,2020-01-01,2020,01,01,0.666667,00:40,0101,2440,20,0.004620
38306,2020-01-01 01:00:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000,3,2020-01-01,2020,01,01,1.000000,01:00,0101,2500,20,0.004013
38307,2020-01-01 01:20:00,20.000000,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,NaN,20.000000,1,2020-01-01,2020,01,01,1.333333,01:20,0101,2520,20,0.004051


## 상품군 결측치 처리

 판매단가가 0 인 상품은 예측에서 제외

In [146]:
def NA_goods(df=X):
    df=df.loc[df['판매단가'] != 0,:]
    df = df.reset_index(drop=True)
    return df

In [147]:
X = NA_goods(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,날짜,년,월,일,시각,시분,월일,시간열,정수노출(분),시청률평균
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,2019-01-01,2019,01,01,6.000000,06:00,0101,600,20,0.002632
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,2019-01-01,2019,01,01,6.000000,06:00,0101,600,20,0.002632
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,2019-01-01,2019,01,01,6.333333,06:20,0101,620,20,0.002434
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,2019-01-01,2019,01,01,6.333333,06:20,0101,620,20,0.002434
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,2019-01-01,2019,01,01,6.666667,06:40,0101,640,20,0.003162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,2019-12-31,2019,12,31,23.666667,23:40,1231,2340,20,0.004741
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,2020-01-01,2020,01,01,0.000000,00:00,0101,2400,20,0.004069
37369,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,2020-01-01,2020,01,01,0.000000,00:00,0101,2400,20,0.004069
37370,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,2020-01-01,2020,01,01,0.000000,00:00,0101,2400,20,0.004069


##  토요일 곁측치 처리

매주 토요일 18:00~18:20은 정보방송시간으로 추정 제외(당사사정에 따라 ±20 편성 변경 있음)

In [148]:
def NA_Saturday(df = X):
    t = ['월','화','수','목','금','토','일']
    r = datetime.datetime.today()
    def printDayOfTheWeek(year, month, day):
        dayOfTheWeek = ['월','화','수','목','금','토','일']
        return dayOfTheWeek[datetime.datetime(year,month,day).weekday()]
    day_list = []
    for date in X.iloc[:,0]:
        date = str(date)
        year = int(date[:4])
        month = int(date[5:7])
        day = int(date[8:10])
        first = printDayOfTheWeek(year,month,day)
        second = date[11:16]
        day_list.append(first+second)
    X["요일/시간"] = day_list
    for i in range(len(df)):
        time = df.loc[i,'요일/시간'] # 여기도 요일/시간 인 loc 로 바꾸었습니다!
        if (time[0:3]=="토18" and int(time[4:])<20):
            df = df.drop(i)
    return df

In [149]:
NA_Saturday(X)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,년,월,일,시각,시분,월일,시간열,정수노출(분),시청률평균,요일/시간
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,2019,01,01,6.000000,06:00,0101,600,20,0.002632,화06:00
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,2019,01,01,6.000000,06:00,0101,600,20,0.002632,화06:00
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,2019,01,01,6.333333,06:20,0101,620,20,0.002434,화06:20
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,2019,01,01,6.333333,06:20,0101,620,20,0.002434,화06:20
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,2019,01,01,6.666667,06:40,0101,640,20,0.003162,화06:40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,...,2019,12,31,23.666667,23:40,1231,2340,20,0.004741,화23:40
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,...,2020,01,01,0.000000,00:00,0101,2400,20,0.004069,수00:00
37369,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,...,2020,01,01,0.000000,00:00,0101,2400,20,0.004069,수00:00
37370,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,...,2020,01,01,0.000000,00:00,0101,2400,20,0.004069,수00:00


In [150]:
#check 하는 용도라 뺴놨습니다!
for i in range(len(NA_Saturday(X))):
    time = X.loc[i,'요일/시간'] # 여기 loc 로 바꾸었습니다. 왜냐하면 앞에서 평균방송분 열을 추가해야되서 열이 하나 늘어났어요
    if time[0:3]=="토18":
        print(time)

토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:30
토18:40
토18:50
토18:20
토18:50
토18:20
토18:30
토18:40
토18:50
토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:20
토18:20
토18:20
토18:20
토18:20
토18:40
토18:40
토18:40
토18:40
토18:40
토18:40


In [151]:
X = NA_Saturday(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,년,월,일,시각,시분,월일,시간열,정수노출(분),시청률평균,요일/시간
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,2019,01,01,6.000000,06:00,0101,600,20,0.002632,화06:00
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,2019,01,01,6.000000,06:00,0101,600,20,0.002632,화06:00
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,2019,01,01,6.333333,06:20,0101,620,20,0.002434,화06:20
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,2019,01,01,6.333333,06:20,0101,620,20,0.002434,화06:20
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,2019,01,01,6.666667,06:40,0101,640,20,0.003162,화06:40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,...,2019,12,31,23.666667,23:40,1231,2340,20,0.004741,화23:40
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,...,2020,01,01,0.000000,00:00,0101,2400,20,0.004069,수00:00
37369,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,...,2020,01,01,0.000000,00:00,0101,2400,20,0.004069,수00:00
37370,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,...,2020,01,01,0.000000,00:00,0101,2400,20,0.004069,수00:00


힘들게 코드를 짰으나 토요일 18:00~18:20에 해당하는 방송은 없었다.............................

## 취급액 50000-> 제거

In [157]:
(X.loc[:,'취급액'] <= 50000).sum()

1993

In [158]:
def NA_50000(df=X):
    for i in range(len(df)):
        if df.loc[i,'취급액'] <= 50000:
            df=df.drop(i)
    df = df.reset_index(drop=True)
    return df

In [159]:
X = NA_50000(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,08:00~10:00,10:00~12:00,12:00~14:00,14:00~16:00,16:00~18:00,18:00~20:00,20:00~02:00,22:00~00:00,00:00~02:00,02:00~04:00
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,0,0,0,0,0,0,0,0,0,0
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,0,0,0,0,0,0,0,0,0,0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,0,0,0,0,0,0,0,0,0,0
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,0,0,0,0,0,0,0,0,0,0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35374,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,...,0,0,0,0,0,0,0,0,0,0
35375,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,...,0,0,0,0,0,0,0,0,0,0
35376,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,...,0,0,0,0,0,0,0,0,0,0
35377,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,...,0,0,0,0,0,0,0,0,0,0


In [160]:
X.shape

(35379, 32)

# 변수 늘리기

## 시간 변수 세분화

특정 시간대에 홈쇼핑 방송 시청이 활발하거나, 홈쇼핑 상품 매출이 높을 것이라 생각하여 2시간으로 세분화한 시간 변수를 사용했다.

//식사시간의 경우 유의미한 데이터가 있을 수 있다. 그 시간대를 잡아서 변수를 늘리면 효과가 있을거같다. <br>
하지만 아침 식사시간의 경우 거르는 인구가 33%(통계청) 이나 되는 이유로, 오히려 혼란을 가중시킬 수 있기떄문에 그냥 점심과 저녁만 고려하자

In [ ]:
hour = X.loc[X['시간열']>2400,'시분']
hour = hour[~hour.duplicated()]
hour = hour.tolist()
X['시간열'].value_counts()

In [167]:
def time_2h(df = X):
    df['06:00~08:00'] = 0
    df['08:00~10:00'] = 0
    df['10:00~12:00'] = 0
    df['12:00~14:00'] = 0
    df['14:00~16:00'] = 0
    df['16:00~18:00'] = 0
    df['18:00~20:00'] = 0
    df['20:00~02:00'] = 0
    df['22:00~00:00'] = 0
    df['00:00~02:00'] = 0
    df['02:00~04:00'] = 0
     
    df.loc[(600<=df['시간열'])&(df['시간열']<800),'06:00~08:00'] = 1
    df.loc[(800<=df['시간열'])&(df['시간열']<1000),'08:00~10:00'] = 1
    df.loc[(1000<=df['시간열'])&(df['시간열']<1200),'10:00~12:00'] = 1
    df.loc[(1200<=df['시간열'])&(df['시간열']<1400),'12:00~14:00'] = 1
    df.loc[(1400<=df['시간열'])&(df['시간열']<1600),'14:00~16:00'] = 1
    df.loc[(1600<=df['시간열'])&(df['시간열']<1800),'16:00~18:00'] = 1
    df.loc[(1800<=df['시간열'])&(df['시간열']<2000),'18:00~20:00'] = 1
    df.loc[(2000<=df['시간열'])&(df['시간열']<2200),'20:00~22:00'] = 1
    df.loc[(2200<=df['시간열'])&(df['시간열']<2400),'22:00~00:00'] = 1
    df.loc[(2400<=df['시간열'])&(df['시간열']<2600),'00:00~02:00'] = 1
    df.loc[(2600<=df['시간열'])&(df['시간열']<2800),'02:00~04:00'] = 1
    return(df)

In [168]:
X = time_2h(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,10:00~12:00,12:00~14:00,14:00~16:00,16:00~18:00,18:00~20:00,20:00~02:00,22:00~00:00,00:00~02:00,02:00~04:00,20:00~22:00
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,0,0,0,0,0,0,0,0,0,NaN
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,0,0,0,0,0,0,0,0,0,NaN
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,0,0,0,0,0,0,0,0,0,NaN
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,0,0,0,0,0,0,0,0,0,NaN
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,0,0,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35374,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,...,0,0,0,0,0,0,1,0,0,NaN
35375,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,...,0,0,0,0,0,0,0,1,0,NaN
35376,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,...,0,0,0,0,0,0,0,1,0,NaN
35377,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,...,0,0,0,0,0,0,0,1,0,NaN


## 강수량 변수

강수량 데이터의 경우 기상청의 기상자료개방포털의 공공데이터를 사용했다.(https://data.kma.go.kr/stcs/grnd/grndRnList.do?pgmNo=69)
강수량이 높으면 집에서 활동하는 인구가 늘어 홈쇼핑 매출에 큰 영향이 있을 것이라 생각했고, 이에 일별 강수량을 고려했다.

In [ ]:
# 외부데이터 불러오기
rain_df=pd.read_csv(directory+'/19rain.csv',header = 7, encoding='CP949')
rain = rain_df.copy()

In [ ]:
rain.head()

In [ ]:
# 날짜에 맞게 강수량 데이터를 병합 
def rain_feature(df = X):
       
    df = pd.merge(df, rain, on="날짜", how= "inner")
    df.drop(['지점'],axis=1,inplace = True)
    return(df)

X = rain_feature(X) ; X

## 온도 변수

In [ ]:
기온 데이터 또한 마찬가지로 기상청의 기상자료개방포털의 공공데이터를 사용했다.(https://data.kma.go.kr/stcs/grnd/grndTaList.do?pgmNo=70)
기온에 따라 특정 상품군의 매출이 달라질 것이라 생각하여 일별 기온(평균기온, 최고기온, 최저기온)을 고려했다.

In [ ]:
# 외부데이터 불러오기
temp_df=pd.read_csv(directory+'/19temp.csv',header = 7, encoding='CP949')
temp = temp_df.copy()

In [ ]:
temp.head()

In [ ]:
# 날짜에 맞게 기온 데이터를 병합
def temp_feature(df = X):
    df = pd.merge(df, temp, on="날짜", how= "inner")
    df.drop(['지점'],axis=1,inplace = True)
    return(df)
X = temp_feature(X) ; X

## 연예인 변수 

In [ ]:
def celebrity(df=X):
    df['연예인']=0
    list_celebrity=["전지현", "팽현숙", "전철우", "강레오", "김선영", "김정배",
                    "김규흔", "이봉원", "오세득", "유귀열", "이경제", "이만기",
                    "이보은", "임성근", "전철우", "최인선", "김병만", "김병지", 
                    "이동수", "서장훈", "송도순", "효재", "천수봉", 
                    "김정문", "숀리", "이정섭", "황성주"]
    for i in list_celebrity:
        df.loc[df.상품명.str.contains(i),'연예인']=1
    return df

In [ ]:
X=celebrity(X); X

In [ ]:
print(X.sum().연예인) #연예인 이름을 사용한 총 데이터 수

## 월급날

In [ ]:
#https://news.sbs.co.kr/news/endPage.do?news_id=N1005505452
#https://m.blog.naver.com/youngkbblog/221714986107

In [ ]:
def income(df = X):
    df['월급날'] = 0
    df.loc[ ((10<=df['일'].astype('float32'))&(df['일'].astype('float32')<=13)) | ((20<=df['일'].astype('float32'))&(df['일'].astype('float32')<= 23)) | ((25<=df['일'].astype('float32')) & (df['일'].astype('float32')<= 28)),'월급날']  = 1
    return(df)

In [ ]:
X=income(X);X

In [ ]:
X.loc[X["일"]=="10", ["월급날"]] #잘 들어갔는지 확인
print(X.sum().월급날) #총 월급날

## 금,토 및 공휴일

In [ ]:
#https://0muwon.com/entry/2019%EB%85%84-%ED%9C%B4%EC%9D%BC-%EA%B3%B5%ED%9C%B4%EC%9D%BC-%EB%8C%80%EC%B2%B4%ED%9C%B4%EC%9D%BC-%ED%99%A9%EA%B8%88%EC%97%B0%ED%9C%B4-%EC%9B%94%EB%B3%84-%EC%99%84%EB%B2%BD%EC%A0%95%EB%A6%AC

In [ ]:
def holiday(df = X):
    df["휴일"] = 0
    holiday_list=[[1,4,5,6,11,12,13,18,19,20,25,26,27],
                  [1,2,3,4,5,6,8,9,10,15,16,17,22,23,24],
                  [1,2,3,8,9,10,15,16,17,22,23,24,29,30,31],
                  [5,6,7,12,13,14,19,20,21,26,27,28],
                  [3,4,5,6,10,11,12,17,18,19,24,25,26,31],
                  [1,2,6,7,8,9,14,15,16,21,22,23,28,29,30],
                  [5,6,7,12,13,14,19,20,21,26,27,28],
                  [2,3,4,9,10,11,15,16,17,18,23,24,25,30,31],
                  [1,6,7,8,12,13,14,15,20,21,22,27,28,29],
                  [3,4,5,6,9,11,12,13,18,19,20,25,26,27],
                  [1,2,3,8,9,10,15,16,17,22,23,24,29,30],
                  [1,6,7,8,13,14,15,20,21,22,25,27,28,29]]
    for i, holiday in enumerate(holiday_list):
        for j in holiday:
            df.loc[ (df["월"].astype("float32")==i+1) & (df["일"].astype("float32")==j),'휴일'] = 1
    return(df)

In [ ]:
X=holiday(X);X

In [ ]:
X.loc[(X["월"].astype("float32")==3) & (X["일"].astype("float32")==1),["휴일"]] #잘 들어갔는지 확인
print(X.sum().휴일) #총 휴일

## 상품군 power

In [ ]:
def goods_power(df = X):
    df["상품군power"] = 0
    for name in ('가구','가전','건강기능','농수축','생활용품','속옷','의류','이미용','잡화','주방','침구'):
        revenue = 0
        runtime = 0
        for i in range(len(df)):
            if df.loc[i,'상품군'] == name:
                revenue += df.loc[i,'취급액']/df.loc[i,'판매단가']
                runtime += df.loc[i,'평균방송분']
            else:
                pass
        for i in range(len(df)):
            if df.loc[i,'상품군'] == name:
                df.loc[i,'상품군power'] = round(revenue/runtime,2)
    
    return(df)

In [ ]:
X = goods_power(X) ; X

## 소비자물가지수

In [ ]:
def CPI(df = X):
    df['CPI'] = 0
    day_list = []
    for date in df.loc[:,'방송일시']:
        year = date[2:4]
        month = date[5:7]
        day_list.append(year+month)
    df['연월'] = day_list
    cpi1,cpi2,cpi3,cpi4,cpi5,cpi6,cpi7,cpi8,cpi9,cpi10,cpi11,cpi12,cpi13 = 0.8, 0.5, 0.4, 0.6, 0.7, 0.7, 0.6, 0.0, -0.4, 0.0, 0.2, 0.7, 1.5
    df.loc[df['연월']== '1901','CPI'] = cpi1
    df.loc[df['연월']== '1902','CPI'] = cpi2
    df.loc[df['연월']== '1903','CPI'] = cpi3
    df.loc[df['연월']== '1904','CPI'] = cpi4
    df.loc[df['연월']== '1905','CPI'] = cpi5
    df.loc[df['연월']== '1906','CPI'] = cpi6
    df.loc[df['연월']== '1907','CPI'] = cpi7
    df.loc[df['연월']== '1908','CPI'] = cpi8
    df.loc[df['연월']== '1909','CPI'] = cpi9
    df.loc[df['연월']== '1910','CPI'] = cpi10
    df.loc[df['연월']== '1911','CPI'] = cpi11
    df.loc[df['연월']== '1912','CPI'] = cpi12
    df.loc[df['연월']== '2001','CPI'] = cpi13
    del df['연월']
    
    return(df)

In [ ]:
X = CPI(X); X

## 소비자 동향지수

In [ ]:
def CSI(df = X):
    df['CSI'] = 0
    day_list = []
    for date in df.loc[:,'방송일시']:
        year = date[2:4]
        month = date[5:7]
        day_list.append(year+month)
    df['연월'] = day_list
    csi1,csi2,csi3,csi4,csi5,csi6,csi7,csi8,csi9,csi10,csi11,csi12,csi13 = 97.5, 99.5, 99.8, 101.6, 97.9, 97.5, 95.9, 92.5, 96.9, 98.6, 101.0, 100.5, 104.2
    df.loc[df['연월']== '1901','CSI'] = csi1
    df.loc[df['연월']== '1902','CSI'] = csi2
    df.loc[df['연월']== '1903','CSI'] = csi3
    df.loc[df['연월']== '1904','CSI'] = csi4
    df.loc[df['연월']== '1905','CSI'] = csi5
    df.loc[df['연월']== '1906','CSI'] = csi6
    df.loc[df['연월']== '1907','CSI'] = csi7
    df.loc[df['연월']== '1908','CSI'] = csi8
    df.loc[df['연월']== '1909','CSI'] = csi9
    df.loc[df['연월']== '1910','CSI'] = csi10
    df.loc[df['연월']== '1911','CSI'] = csi11
    df.loc[df['연월']== '1912','CSI'] = csi12
    df.loc[df['연월']== '2001','CSI'] = csi13
    del df['연월']
    
    return(df)

In [ ]:
X = CSI(X); X

## 소매업태별(홈쇼핑) 판매액지수(불변지수)

In [ ]:
def RS(df = X):
    df['RS'] = 0
    day_list = []
    for date in df.loc[:,'방송일시']:
        year = date[2:4]
        month = date[5:7]
        day_list.append(year+month)
    df['연월'] = day_list
    x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13 = 139.3, 113.3, 134.0, 130.1, 133.8, 123.4, 124.7, 120.3, 124.3, 139.0, 138.8, 138.0, 131.7
    df.loc[df['연월']== '1901','RS'] = x1
    df.loc[df['연월']== '1902','RS'] = x2
    df.loc[df['연월']== '1903','RS'] = x3
    df.loc[df['연월']== '1904','RS'] = x4
    df.loc[df['연월']== '1905','RS'] = x5
    df.loc[df['연월']== '1906','RS'] = x6
    df.loc[df['연월']== '1907','RS'] = x7
    df.loc[df['연월']== '1908','RS'] = x8
    df.loc[df['연월']== '1909','RS'] = x9
    df.loc[df['연월']== '1910','RS'] = x10
    df.loc[df['연월']== '1911','RS'] = x11
    df.loc[df['연월']== '1912','RS'] = x12
    df.loc[df['연월']== '2001','RS'] = x13
    del df['연월']
    
    return(df)

In [ ]:
X = RS(X); X

## 심야 시간대 표현

00 시 이후는 24를 더해주었습니다.

In [ ]:
X.loc[X.loc[:,'시각'] < 3,'시각'] = X.loc[X.loc[:,'시각'] < 3,'시각'] + 24

In [ ]:
X

In [ ]:
X.columns

## Prime Time

In [ ]:
# https://www.etnews.com/20180718000049?m=1

In [ ]:
## one-hot vector(프라임타임 or not)

def prime_time(df = X):
    df['PrimeTime'] = 0
    
    for i in range(len(df)):
        if (9 <= df.loc[i,'시각']) and (df.loc[i,'시각'] < 11):
            df.loc[i,'PrimeTime'] = 1
        elif df.loc[i,'상품군']=="농수축" and (16<=df.loc[i,'시각']) and (df.loc[i,'시각']< 18):
            df.loc[i,'PrimeTime'] = 1            
        elif df.loc[i,'상품군']!="농수축" and (20<=df.loc[i,'시각']) and (df.loc[i,'시각']< 23):
            df.loc[i,'PrimeTime'] = 1
    
    return df

In [ ]:
prime_time(X) ;X

In [ ]:
X['PrimeTime'].sum()
## 생각보다 해당되는 항목이 많네요..!!
## 시간 처리할 때 ( , ) 이 아니라 [ , ) 로 했습니다.

## 남성, 여성 상품

In [ ]:
def gender(df=X):
    df["남성상품"]=0
    df["여성상품"]=0
    list_male=["남성","드로즈","푸마 드라이셀라인 뉴 런닝","언더셔츠","카파 런닝","리복 스피드윅 런닝",
               "트렁크","프로스펙스 2020 퍼포먼스 런닝"]
    list_female=["여성","브라","밍크","팬티","치마","레깅스","란쥬","핸드백",
                 "앙상블","엔셀라두스 밴딩팬츠","레이스","엘리자베스아덴","버켓 백",
                 "크로스바디 백","립스틱","쿠션","사첼백","숄더","속눈썹","토트백",
                 "원피스","틴트","투웨이백","이지탑","크로스백","클러치백","보정런닝",
                 "오렐리안 히트모 스킨컬렉션 기모런닝","보정 런닝"]
    for i in ["의류","속옷","잡화","이미용"]:
        for j in list_male:
            df.loc[(df["상품군"]==i)&(df.상품명.str.contains(j)),"남성상품"]=1
        for k in list_female:
            df.loc[(df["상품군"]==i)&(df.상품명.str.contains(k)),"여성상품"]=1
    return df

In [ ]:
X = gender(X); X

## 무이자, 일시불 처리

무이자, 일시불과 같은 소비 방식 또한 소비에 영향을 미칠 것이라 판단하여 상품 별 무이자, 일시불을 나타내는 열을 추가했다.

In [ ]:
def pay(df=X):
    df["무이자"]=0 
    df["일시불"]=0
    pay_in_inst=["무)", "무이자"] # 무), (무), 무이자 등의 형태로 무이자 상품이 표기
    pay_in_full=["일)", "일시불"] # 일), (일), 일시불 등의 형태로 무이자 상품이 표기
    for i in pay_in_inst:
        df.loc[(df.상품명.str.contains(i,regex=False))&(~df.상품명.str.contains("보험")),"무이자"]=1 # (무)로 표기된 무배당 보험상품 제외
    for j in pay_in_full:
        df.loc[(df.상품명.str.contains(j, regex=False))&(~df.상품명.str.contains("세일")),"일시불"]=1 # (세일)로 표기된 세일 상품 제외
    return df

In [ ]:
X = pay(X); X

In [ ]:
# 확인용
X.loc[X['무이자']==1, '상품명']

## 유명 기업/브랜드명

In [ ]:
def brandname(df=X):
    df['유명기업/브랜드']=0
    lst=["LG","SPC삼립","경남제약", "광동", "구찌", "굿프렌드", "농협", "딤채",
         "락앤락","리복","마리끌레르","뱅뱅","베지밀","보루네오","본죽","삼성",
         "엘르","종근당건강","쿠쿠",'쿠첸',"푸마","프라다","하림","한샘","버버리",
         "비비고"]
    for i in lst:
        df.loc[df.상품명.str.contains(i),'유명기업/브랜드']=1
    for i in range(len(df)):
        if ("폴로" in df.loc[i,"상품명"]) or ("USPA" in df.loc[i,"상품명"]):
            df.loc[i,"상품명"] = 1
            
    return df

In [ ]:
X = brandname(X); X

# 저장

In [ ]:
X.columns

In [ ]:
X.to_excel('X0819.xlsx')

In [ ]:
weekday.to_csv('weekday.csv')
weekend.to_csv('weekend.csv')